<a href="https://colab.research.google.com/github/SkyHaines/IMLO/blob/main/Coursework%3F%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The CIFAR 10 dataset contains 60,000 3x32x32 colour images, with 10 classes.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

# Need to look into how this works, perhaps i could tweak it to improve accuracy?
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Download training and testing data
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transform
)

Now, I need to experiment with batch size. (The number of images processed before updating the weights).

As I understand it,
Larger batch size : Faster training, lower accuracy
Smaller batch size: Slower training, higher accuracy

So I'm going to do some trial and error I think, to get the batch size at a point where I use up just under the whole 4 training hours, with the highest accuracy :)

Contextualise: There are 50,000 training imgs, so I'm going to try a batch size of 100 for now? (As its a multiple, I assume it helps somehow?)
UPDATED: Changed to 64, research suggests it helps with runtime to have it as a power of 2, for parallelism. Will take this into account for further optimisation.

In [2]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

classes = ('airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck')

Now I need to define the nn architecture/structure? I think I need to use a CNN rather than a regular NN because it's for images rather than numerical data

In [3]:
device = "cpu"
class MyCNN(nn.Module):
  def __init__(self):
    super().__init__()

    # First layer? Has 3 input channels due to RGB
    # input tensor = size of prev layer output * filters in prev layer
    # output tensor = size of input img * filters
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=2, stride=1, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels= 12,kernel_size=2, stride=1, padding=1)
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    self.fc1 = nn.Linear(in_features=12*8*8, out_features=128)
    self.fc2 = nn.Linear(in_features=128, out_features=10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool2(x)
    x = torch.flatten(x,1)
    x = self.fc1(x)
    x = self.fc2(x)
    return x

model = MyCNN().to(device)
print(model)

MyCNN(
  (conv1): Conv2d(3, 6, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=768, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [4]:
loss = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=0.001)

In [5]:
def train(dataloader, model, loss_fn, optimiser):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      optimiser.zero_grad()

      pred = model(X)
      loss = loss_fn(pred, y)
      loss.backward()
      optimiser.step()

      if batch % 100 == 0:
        loss, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
epochs = 25
for t in range(epochs):
  print(f"Epoch {t+1} \n-------------------------------")
  train(train_dataloader, model, loss, optimiser)
  test(test_dataloader, model, loss)
print("Done :)")

Epoch 1 
-------------------------------
loss: 2.316995  [   64/50000]
loss: 2.297624  [ 6464/50000]
loss: 2.306863  [12864/50000]
loss: 2.291114  [19264/50000]
loss: 2.315272  [25664/50000]
loss: 2.304641  [32064/50000]
loss: 2.294323  [38464/50000]
loss: 2.302375  [44864/50000]
Test error: 
 Accuracy: 11.4%, Avg loss: 2.300893 

Epoch 2 
-------------------------------
loss: 2.304655  [   64/50000]
loss: 2.304469  [ 6464/50000]
loss: 2.301111  [12864/50000]
loss: 2.298254  [19264/50000]
loss: 2.296534  [25664/50000]
loss: 2.290886  [32064/50000]
loss: 2.300788  [38464/50000]
loss: 2.297734  [44864/50000]
Test error: 
 Accuracy: 12.6%, Avg loss: 2.295924 

Epoch 3 
-------------------------------
loss: 2.288890  [   64/50000]
loss: 2.292371  [ 6464/50000]
loss: 2.293914  [12864/50000]
loss: 2.300727  [19264/50000]
loss: 2.291284  [25664/50000]
loss: 2.292559  [32064/50000]
loss: 2.284434  [38464/50000]
loss: 2.285740  [44864/50000]
Test error: 
 Accuracy: 14.6%, Avg loss: 2.289032 

E